In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sksurv.util import Surv
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest, GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

from joblib import dump
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(exist_ok=True, parents=True)

df = pd.read_parquet(DATA_DIR / "scania_processed.parquet")
df.head(), df.shape



(        171_0     666_0     837_0    167_0      167_1       167_2       167_3  \
 0  10189950.0  372685.0   41670.0      0.0   811605.0   1644061.0    560460.0   
 1   5648790.0  289371.0   68717.0  10415.0  9137870.0  74655621.0  45991626.0   
 2   7603590.0  230831.0  100121.0   5918.0  8225139.0  17004223.0  10504195.0   
 3   4842780.0  210381.0  152385.0   7128.0  4342398.0  13348382.0  11538870.0   
 4   6623040.0  280531.0  164673.0      0.0   513466.0    693990.0    350295.0   
 
         167_4       167_6      167_7  ...  Spec_7_Cat0  Spec_7_Cat1  \
 0    873435.0         0.0        0.0  ...         True        False   
 1  65888583.0   5813857.0     3351.0  ...        False         True   
 2  30792854.0  27370868.0  3819383.0  ...        False         True   
 3  30085352.0  53625651.0  1225967.0  ...        False         True   
 4    735390.0       570.0        0.0  ...        False         True   
 
    Spec_7_Cat2  Spec_7_Cat4  Spec_7_Cat5  Spec_7_Cat6  Spec_7_Cat7  RUL

In [2]:
feature_cols = [c for c in df.columns if c not in ["RUL", "in_study_repair", "vehicle_id"]]

X = df[feature_cols]
y_time  = df["RUL"].astype(float)
y_event = df["in_study_repair"].astype(bool)

X_train, X_test, y_time_train, y_time_test, y_event_train, y_event_test = \
    train_test_split(
        X, y_time, y_event,
        test_size=0.2,
        random_state=12345,
        stratify=y_event
    )

y_train = Surv.from_arrays(event=y_event_train.to_numpy(),
                           time=y_time_train.to_numpy())
y_test = Surv.from_arrays(event=y_event_test.to_numpy(),
                          time=y_time_test.to_numpy())

X_train.shape, X_test.shape


((18840, 76), (4710, 76))

In [3]:
cph = CoxPHSurvivalAnalysis()
cph.fit(X_train, y_train)

pred_train_cph = cph.predict(X_train)
pred_test_cph  = cph.predict(X_test)

cph_c_train = concordance_index_censored(y_event_train, y_time_train, pred_train_cph)[0]
cph_c_test  = concordance_index_censored(y_event_test, y_time_test, pred_test_cph)[0]

print("CPH C-index train:", cph_c_train)
print("CPH C-index test :", cph_c_test)


e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=3.63911e-21): result may not be accurate.
  delta = solve(
e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=9.16553e-22): result may not be accurate.
  delta = solve(
e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=3.20093e-22): result may not be accurate.
  delta = solve(
e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=5.49076e-22): result may not be accurate.
  delta = solve(
e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=2.04779e-22): result may

CPH C-index train: 0.6855111429596336
CPH C-index test : 0.6475524641836975


e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sksurv\linear_model\coxph.py:492: LinAlgWarning: Ill-conditioned matrix (rcond=9.33668e-27): result may not be accurate.
  delta = solve(


In [4]:
rsf = RandomSurvivalForest(
    n_estimators=200,
    random_state=12345,
    n_jobs=-1,
    oob_score=True
)

param_dist_rsf = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": [5, 15, 30, 100, 250],
    "min_samples_split": [10, 15, 20, 30, 50],
    "min_samples_leaf": [5, 10, 20],
}

rsf_cv = RandomizedSearchCV(
    rsf,
    param_distributions=param_dist_rsf,
    n_iter=15,
    cv=5,
    n_jobs=-1,
    random_state=12345
)

rsf_cv.fit(X_train, y_train)
print("Best RSF params:", rsf_cv.best_params_)
rsf_best = rsf_cv.best_estimator_

pred_train_rsf = rsf_best.predict(X_train)
pred_test_rsf  = rsf_best.predict(X_test)

rsf_c_train = concordance_index_censored(y_event_train, y_time_train, pred_train_rsf)[0]
rsf_c_test  = concordance_index_censored(y_event_test, y_time_test, pred_test_rsf)[0]

print("RSF C-index train:", rsf_c_train)
print("RSF C-index test :", rsf_c_test)



e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
5 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\joblib\_utils.py", line 72, in __call__
    return self.func(**kwargs)
  File "e:\work\project\scania_rul_survival\scania_rul_env\lib\site-packages\joblib\parallel.py", line 607, in __call__
    return [func(*args, **kwargs) for func, args, kwargs in self.items]
  File "e:\work\project\scania_rul_survival\scania_rul

Best RSF params: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': 30}
RSF C-index train: 0.950728202903198
RSF C-index test : 0.7167915116191169


In [5]:
gbsa = GradientBoostingSurvivalAnalysis(random_state=12345)

param_dist_gb = {
    "n_estimators": [50, 100, 200],
    "min_samples_split": [30, 50],
    "min_samples_leaf": [5, 10, 20],
    "learning_rate": [0.5, 1.0],
    "max_depth": [1, 5, 6],
}

gbsa_cv = RandomizedSearchCV(
    gbsa,
    param_distributions=param_dist_gb,
    n_iter=15,
    cv=5,
    n_jobs=-1,
    random_state=12345
)

gbsa_cv.fit(X_train, y_train)
print("Best GBSA params:", gbsa_cv.best_params_)
gbsa_best = gbsa_cv.best_estimator_

pred_train_gb = gbsa_best.predict(X_train)
pred_test_gb  = gbsa_best.predict(X_test)

gbsa_c_train = concordance_index_censored(y_event_train, y_time_train, pred_train_gb)[0]
gbsa_c_test  = concordance_index_censored(y_event_test, y_time_test, pred_test_gb)[0]

print("GBSA C-index train:", gbsa_c_train)
print("GBSA C-index test :", gbsa_c_test)



Best GBSA params: {'n_estimators': 100, 'min_samples_split': 30, 'min_samples_leaf': 20, 'max_depth': 6, 'learning_rate': 0.5}
GBSA C-index train: 0.8862393195097956
GBSA C-index test : 0.706968412344234


In [5]:
dump(rsf_best, MODELS_DIR / "rsf_best.joblib")


['E:\\work\\project\\scania_rul_survival\\models\\rsf_best.joblib']